# Generators & Comprehensions - Part 3

This will cover more advanced generators and comprehensions (and other iterator) topics, but it starts with some review (and tips).

In [1]:
# Just mapping (no filtering).
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [2]:
# Mapping and filtering.
[x**2 for x in range(10) if x % 3 != 0]

[1, 4, 16, 25, 49, 64]

In [3]:
# Filtering. (Technically also mapping.)
[x for x in range(10) if x % 3 != 0]

[1, 2, 4, 5, 7, 8]

In [4]:
s = ['hello', 'my', 'name', 'is', 'David', 'Vassallo']

In [5]:
# A list of the all-lowercase words in s.
# Only filters.
[word for word in s if word.islower()]

['hello', 'my', 'name', 'is']

In [6]:
# A list of the lengths of all the words in s.
# Only maps.
[len(word) for word in s]

[5, 2, 4, 2, 5, 8]

In [7]:
# A list of the lengths of the all-lowercase words in s
# Maps and filters. 
[len(word) for word in s if word.islower()]

[5, 2, 4, 2]

In [8]:
# A list of the lengths of the all-lowercase words in s, using map and filter.
# This does not use any comprehensions of any kind.
list(map(len, filter(str.islower, s)))

[5, 2, 4, 2]

In [9]:
# Use LBYL to show that map and filter are iterator types.
# Then likewise show, with instances of them, that they are iterator objects.
from collections.abc import Iterator, Mapping

In [10]:
issubclass(map, Iterator)

True

In [11]:
issubclass(filter, Iterator)

True

In [12]:
isinstance(map(len, s), Iterator)

True

In [13]:
isinstance(filter(str.islower, s), Iterator)

True

In [14]:
# The builtin map type should not be confused with mappings (i.e., dict-like objects).
# Show with LBYL that map is not a mapping type.
issubclass(map, Mapping)

False

**Tip 1:** Iterators are iterable. That is, you can call `next` on iterators, but you can also call `iter` on an iterator, which gives you back an equivalent iterator, almost always the very same iterator object.

**Tip 2:** Do not confuse callables with iterators. A callable gives a value when called; an iterator gives a value when passed to `next`.

**Tip 3:** Strongly prefer high-level constructs like `for` over explicitly calling `iter` or `next`.

It is fairly rare to call `next` outside of code that performs general iterator operations such as those performed by the code in `itertools`.

You shouldn't be afraid to use the `iter` and `next` builtin (in any context). But every time you use them, you should be able to clearly articulate why you need or want to use them instead of higher level constructs like `for`.

**Tip 4:** Anytime you write something like:

```python
(x for x in some_expression)
```

Or:

```python
((x, y) for x, y in some_expression)
```

Make sure you understand why you are writing that instead of just:

```python
some_expression
```

Only in rare cases should you write comprehensions of that form.

**Tip 5:** Remember the `sum` builtin.

**Tip 6:** Materialize an iterable when *both*:

1. it might be consumed by iteration (in almost all cases, this is when it's an iterator) *and*

2. you need, or may need, to iterate through its values multiple times.

You should not usually perform materialization unless both of these conditions hold.

Note also that `a = list(b)` and `a = tuple(b)` materialize `b`, but `a = b` does not, because assignment just copies a reference. There is no situation in which running `a = b` and then iterating through `a` behaves any differently from just iterating through `b`.



## `finally` blocks in generators

In [15]:
def gen(): 
    try: 
        yield 1
        yield 2
        yield 3
    finally: 
        print('Done.')

In [16]:
[x for x in gen()] # don't do this

Done.


[1, 2, 3]

In [17]:
list(gen()) # do this instead

Done.


[1, 2, 3]

In [18]:
def f(): 
    gen()

In [19]:
f()

In [20]:
def f2(): 
    g = gen()
    return next(g)

In [21]:
f2()

Done.


1

## `GeneratorExit` is raised when destroying suspended generators

In [22]:
print(GeneratorExit.__doc__)

Request that a generator exit.


In [23]:
GeneratorExit.__bases__

(BaseException,)

In [24]:
ValueError.__bases__

(Exception,)

In [25]:
TypeError.__bases__

(Exception,)

In [26]:
StopIteration.__bases__

(Exception,)

In [27]:
SystemExit.__bases__

(BaseException,)

In [28]:
Exception.__bases__

(BaseException,)

In [29]:
def gen2(): 
    try: 
        yield 1
        yield 2
        yield 3
    except GeneratorExit: 
        print('Interrupted.')
    finally: 
        print('Done.')

In [30]:
list(gen2())

Done.


[1, 2, 3]

In [31]:
def f3(): 
    g = gen2()
    return next(g)

In [32]:
f3()

Interrupted.
Done.


1

## It is an error to yield after `GeneratorExit`

In [33]:
def gen_bad(): 
    try: 
        yield 1
        yield 2
        yield 3
    except GeneratorExit: 
        print('Interrupted.')
        yield 4
    finally: 
        print('Done.')

In [34]:
def f4(): 
    g = gen_bad()
    return next(g)

In [35]:
f4()

Exception ignored in: <generator object gen_bad at 0x000001A2907AF6F0>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_23168\1205670228.py", line 1, in <cell line: 1>
RuntimeError: generator ignored GeneratorExit


Interrupted.


1

In [36]:
# s = [1]
# s.append(s)

## Generator objects with reference cycles

In [37]:
def gen_rcycle():     
    try: 
        s = [1]
        yield s
    finally: 
        print('Done')

In [38]:
def do_stuff(): 
    ob = gen_rcycle() 
    next(ob).append(ob)

In [39]:
do_stuff()

In [40]:
import gc

In [41]:
gc.collect()

Done


235

## Generator objects are not context managers

In [42]:
def do_more_stuff(): 
    ob = gen_rcycle() 
    next(ob).append(ob)
    return ob

In [54]:
with do_more_stuff(): 
    print('Use resource acquired')

Done
Done


AttributeError: __enter__

In [44]:
gc.collect()

465

## Generator objects do have a `close` method

In [45]:
ob1 = do_more_stuff()

In [46]:
type(ob1)

generator

In [48]:
dir(ob1)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [49]:
help(ob1.close)

Help on built-in function close:

close(...) method of builtins.generator instance
    close() -> raise GeneratorExit inside generator.



In [53]:
ob1 = do_more_stuff()
try: 
    print('If I use ob1, I would use it here')
finally: 
    ob1.close()

If I use ob1, I would use it here
Done


In [55]:
import contextlib

In [56]:
with contextlib.closing(do_more_stuff()): 
    print('Use resource acquired')

Use resource acquired
Done


**TODO:** Examine materialization and acquision/release in `itertools.tee`.

## Review of materialization

In [57]:
def product_three(one, two, three): 
    my_one = list(one)
    my_two = list(two)
    my_three = list(three)
    return ((x, y, z) for x in my_one for y in my_two for z in my_three)

In [58]:
groups = product_three(['a', 'b', 'c'], (x for x in range(4)), (100, 200, 300))

In [59]:
list(groups)

[('a', 0, 100),
 ('a', 0, 200),
 ('a', 0, 300),
 ('a', 1, 100),
 ('a', 1, 200),
 ('a', 1, 300),
 ('a', 2, 100),
 ('a', 2, 200),
 ('a', 2, 300),
 ('a', 3, 100),
 ('a', 3, 200),
 ('a', 3, 300),
 ('b', 0, 100),
 ('b', 0, 200),
 ('b', 0, 300),
 ('b', 1, 100),
 ('b', 1, 200),
 ('b', 1, 300),
 ('b', 2, 100),
 ('b', 2, 200),
 ('b', 2, 300),
 ('b', 3, 100),
 ('b', 3, 200),
 ('b', 3, 300),
 ('c', 0, 100),
 ('c', 0, 200),
 ('c', 0, 300),
 ('c', 1, 100),
 ('c', 1, 200),
 ('c', 1, 300),
 ('c', 2, 100),
 ('c', 2, 200),
 ('c', 2, 300),
 ('c', 3, 100),
 ('c', 3, 200),
 ('c', 3, 300)]